In [62]:
import requests
import pandas as pd
import urllib.parse
import numpy as np
import re

In [63]:
make = 'audi'
model = 'a3'
yearMin = 2010
yearMax = 2020
kmMin = 0
kmMax = 100000
priceMin = 0
priceMax = 10000

'https://api.walla.com/api/v3/cars/search?filters_source=suggester&keywords=audi%20a4&category_ids=100&longitude=-3.69196&latitude=40.41956'
url_base = 'https://api.wallapop.com/api/v3/cars/search?'

parametros = {
    'filters_source': 'suggester',
    'keywords': make + ' ' + model,
    'category_ids': 100,
    'longitude': -3.69196,
    'latitude': 40.41956,
    'yearMin': yearMin,
    'yearMax': yearMax,
    'kmMin': kmMin,
    'kmMax': kmMax,
    'priceMin': priceMin,
    'priceMax': priceMax
}
url_completa = url_base + urllib.parse.urlencode(parametros)
response = requests.get(url_completa)
if response.ok:
    try:
        data = response.json()
        rows = []
        for obj in data['search_objects']:
            row = obj['content']
            rows.append(row)

        df = pd.DataFrame(rows)
    except Exception as e:
        print("Error:", e)


In [64]:
df = df[['title', 'price', 'km', 'engine', 'gearbox', 'horsepower']]

def scrape_year(title):
    # Using regular expression to get a 4-digit combination starting with 1 or 2
    pattern = r'\b(?:1|2)\d{3}\b'
    match = re.search(pattern, title)
    if match:
        year = match.group(0)
        return int(year)
    else:
        return None


df['year'] = df['title'].apply(scrape_year)
df = df.dropna()
df['environmental_label'] = np.nan
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, 3 to 39
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   title                16 non-null     object 
 1   price                16 non-null     float64
 2   km                   16 non-null     float64
 3   engine               16 non-null     object 
 4   gearbox              16 non-null     object 
 5   horsepower           16 non-null     float64
 6   year                 16 non-null     float64
 7   environmental_label  0 non-null      float64
dtypes: float64(5), object(3)
memory usage: 1.1+ KB


In [65]:
import urllib.parse
import requests
import pandas as pd
import numpy as np

In [66]:
make = 'audi'
model = 'a3'
yearMin = 2010
yearMax = 2020
kmMin = 0
kmMax = 100000
priceMin = 0
priceMax = 100000

parametros = {
    'filters_source': 'suggester',
    'keywords': make + ' ' + model,
    'category_ids': 100,
    'longitude': -3.69196,
    'latitude': 40.41956,
    'yearMin': yearMin,
    'yearMax': yearMax,
    'kmMin': kmMin,
    'kmMax': kmMax,
    'priceMin': priceMin,
    'priceMax': priceMax
}

url_base = 'https://api.wallapop.com/api/v3/cars/search?'
url_completa = url_base + urllib.parse.urlencode(parametros)
response = requests.get(url_completa)
print(response)
df_walla = pd.DataFrame()
if response.ok:
    try:
        data = response.json()
        rows = []
        for obj in data['search_objects']:
            row = obj['content']
            rows.append(row)
        df_walla = pd.DataFrame(rows)
    except Exception as e:
        print("Error:", e)
print(df.columns)

<Response [200]>
Index(['title', 'price', 'km', 'engine', 'gearbox', 'horsepower', 'year',
       'environmental_label'],
      dtype='object')


In [67]:
parametros = {
        'tipo_busqueda': '2',
        'seminuevo': '0',
        'ord[]': 'marca_up',
        'searched3': '',
        'color': '',
        'combustible_id': '',
        'precio_desde': priceMin,
        'precio_hasta': priceMax,
        'scf_fee_desde': '',
        'scf_fee_hasta': '',
        'potencia_desde': '',
        'potencia_hasta': '',
        'km_desde': kmMin,
        'km_hasta': kmMax,
        'anyo_desde': yearMin,
        'anyo_hasta': yearMax,
        'cambio': '',
        'puertas': '',
        'plazas': '',
        'vendedor': '',
        'make_list[]': '',
        'transmission_name': '',
        'agent_type_name': '',
        'has_financing': '',
        'reservable': '',
        'search3': make + ' ' + model,
    }
url_base = 'https://www.coches.com/api/vo/pills/?'
url_completa = url_base + urllib.parse.urlencode(parametros)
response = requests.get(url_completa)
if response.ok:
    try:
        data = response.json()
        df_com = pd.DataFrame(data['pills'])
    except:
        print("Error:", e)
    print(df.columns)
    print(len(df))

Index(['title', 'price', 'km', 'engine', 'gearbox', 'horsepower', 'year',
       'environmental_label'],
      dtype='object')
16


In [69]:
same_columns_wallapop = ['id', 'title', 'images', 'price', 'brand', 'model', 'year', 'km', 'engine', 'horsepower']
same_columns_coches_com = ['id','image', 'price', 'url' , 'make', 'model' , 'fuel', 'cv', 'km', 'year']
comon_names = ['id','image','price','make','model','year','km','fuelType','horsepower','cv','url']
rename_columns = {
    'engine': 'fuelType',
    'fuel' : 'fuelType',
    'horsepower': 'cv',
    'brand' : 'make',
    'images': 'image',
    'url': 'link'
}
delete_columns = ['title']
df_walla['images'] = df_walla['images'].apply(lambda x: x[0]['original'] if isinstance(x, list) and len(x) > 0 else None)
df_walla.rename(columns=rename_columns, inplace=True)

df_walla.drop(columns=delete_columns, inplace=True, errors='ignore')

df_com.rename(columns=rename_columns, inplace=True)

df_com.drop(columns=delete_columns, inplace=True, errors='ignore')

df_walla = df_walla[[col for col in comon_names if col in df_walla.columns]]
df_com = df_com[[col for col in comon_names if col in df_com.columns]]

df_walla['site'] = 'walla'

df_com['site'] = 'com'


In [72]:
print(len(df_walla))
df_walla.head(1)

40


,id,image,price,make,model,year,km,fuelType,cv,site
0,3zl88w3m546x,https://cdn.wallapop.com/images/10420/gj/wi/__...,5500.0,Audi,A3 2.0 FSI,2004,218000.0,Gasoline,NaN,walla


In [73]:
print(len(df_com))
df_com.head(1)

20


,id,image,price,make,model,year,km,fuelType,cv,site
0,8125894,https://images.coches.com/_vo_/fotos/usados/20...,26.900 €,Audi,A3,2014,75.000,Gasolina,300,com
